Make a copy of this notebook and rename using your USERID in the following format, 2017CSZ8058

Give read access to keshavkolluru@gmail.com, vishalsaley114@gmail.com and kartikeya.badola@gmail.com


In [7]:
# ## DONT CHANGE THIS CELL
# !wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip

In [2]:
## Import relevant packages

import os

import time
import datetime

import torch
import torch.optim as O
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

import logging
from argparse import ArgumentParser

from pdb import set_trace

In [23]:
import pandas
x_train = pandas.read_csv('./data/train/train.data.txt', delimiter='\t', header=None, names=['word', 'POS', 'position', 'sen1', 'sen2'])
y_train = pandas.read_csv('./data/train/train.gold.txt', header=None, names=['label'])
train_data = pandas.concat((x_train, y_train), axis=1)
print(train_data[:4])
print(train_data.POS.unique(), train_data.label.unique())

    word POS position                                            sen1  \
0  carry   V      2-1              You must carry your camping gear .   
1     go   V      2-6  Messages must go through diplomatic channels .   
2  break   V      0-2                                Break an alibi .   
3    cup   N      8-4         He wore a jock strap with a metal cup .   

                                                sen2 label  
0                    Sound carries well over water .     F  
1   Do you think the sofa will go through the door ?     F  
2  The wholesaler broke the container loads into ...     F  
3            Bees filled the waxen cups with honey .     T  
['V' 'N'] ['F' 'T']
